In [68]:
import requests

def get_weather_data():
  points = ['40.0231,-79.2988', '28.1946971,-82.3459212']
  for point in points:
    response = requests.get('https://api.weather.gov/points/' + point)
    params = response.json()
    forecast_url = params['properties']['forecast']
    forecast = requests.get(forecast_url)
    forecast_data = forecast.json()
    temperature = forecast_data['properties']['periods'][0]['temperature']
    forecast_blurb = forecast_data['properties']['periods'][0]['shortForecast']
    print(temperature, forecast_blurb)
  return temperature, forecast_blurb,forecast_data

get_weather_data()



37 Chance Showers And Thunderstorms then Mostly Cloudy
81 Slight Chance Showers And Thunderstorms


(81,
 'Slight Chance Showers And Thunderstorms',
 {'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
   {'@version': '1.1',
    'wx': 'https://api.weather.gov/ontology#',
    'geo': 'http://www.opengis.net/ont/geosparql#',
    'unit': 'http://codes.wmo.int/common/unit/',
    '@vocab': 'https://api.weather.gov/ontology#'}],
  'type': 'Feature',
  'geometry': {'type': 'Polygon',
   'coordinates': [[[-82.3663422, 28.2165244],
     [-82.36874970000001, 28.193820600000002],
     [-82.3430005, 28.1916979],
     [-82.3405882, 28.214401300000002],
     [-82.3663422, 28.2165244]]]},
  'properties': {'updated': '2023-01-04T19:44:04+00:00',
   'units': 'us',
   'forecastGenerator': 'BaselineForecastGenerator',
   'generatedAt': '2023-01-04T22:21:48+00:00',
   'updateTime': '2023-01-04T19:44:04+00:00',
   'validTimes': '2023-01-04T13:00:00+00:00/P7DT18H',
   'elevation': {'unitCode': 'wmoUnit:m', 'value': 18.8976},
   'periods': [{'number': 1,
     'name': 'This Afternoon',
   

This first function just verifies that the API is function and returning readable JSON data

The second function provides a thermal sorting algorithm, using state-of-the-art NLP designed in-house by our team of engineers to determine if the provided {lat}{long} pairing is a 'Shred', 'May Shred', or 'No Shred'

In [158]:
list_of_shred = []
list_may_shred = []
list_no_shred = []
list_of_cities = []


def temperature_analysis(list_of_shred = [],list_may_shred = [],list_no_shred = [],list_of_cities = []):
  points = ['40.0231,-79.2988',
  '46.517935737082844,-87.40349091678263',
  '40.645204305328775,-111.49275194259786',
  '38.93871383768245,-119.98104607434624',
  '58.3022283334697,-134.4193242885098',
  '48.80872506257114,-121.70317234117626']
  for point in points:
    response = requests.get('https://api.weather.gov/points/' + point)
    params = response.json()
    forecast_url = params['properties']['forecast']
    city = params['properties']['relativeLocation']['properties']['city']
    list_of_cities.append(city)
    forecast = requests.get(forecast_url)
    forecast_data = forecast.json()
    temperature = forecast_data['properties']['periods'][0]['temperature']
    forecast_blurb = forecast_data['properties']['periods'][0]['shortForecast']
    forecast_detail = forecast_data['properties']['periods'][0]['detailedForecast']
    if temperature >= 25 and temperature < 40:
      if forecast_detail.find("snow") != -1 or forecast_detail.find("Snow") != -1:
        list_of_shred.append(point)
      if forecast_detail.find("sun") != -1:
        list_may_shred.append(point)
      elif forecast_detail.find("cloud") != -1:
        list_no_shred.append(point)
      else:
        list_may_shred.append(point)
    elif temperature < 25:
      list_no_shred.append(point)
    else:
      list_may_shred.append(point)
  return list_of_shred, list_may_shred, list_no_shred,


temperature_analysis()

(['46.517935737082844,-87.40349091678263',
  '58.3022283334697,-134.4193242885098'],
 ['46.517935737082844,-87.40349091678263',
  '40.645204305328775,-111.49275194259786',
  '38.93871383768245,-119.98104607434624',
  '58.3022283334697,-134.4193242885098'],
 ['40.0231,-79.2988', '48.80872506257114,-121.70317234117626'])

Third Function outputs the {lat}{long} pairings selected by the user to a Pandas dataframe to matchup the pairing with the closest City. This dataframe also displays the temperature of the pairing/city. 

In [157]:
import pandas as pd

def list_to_pandas():
    list_of_cities = []
    list_of_temps = []
    points = [
        '40.0231,-79.2988',
        '46.5179,-87.4034',
        '40.6452,-111.4927',
        '38.9387,-119.9810',
        '58.3022,-134.41932',
        '48.8087,-121.7031']
    for point in points:
        response = requests.get('https://api.weather.gov/points/' + point)
        params = response.json()
        forecast_url = params['properties']['forecast']
        city = params['properties']['relativeLocation']['properties']['city']
        forecast = requests.get(forecast_url)
        forecast_data = forecast.json()
        list_of_cities.append(city)
        list_of_temps.append(forecast_data['properties']['periods'][0]['temperature'])
    df = pd.DataFrame(list_of_cities)
    df.columns = ['City']
    df.insert(1, "Loc", points)
    df.insert(2, "Temp", list_of_temps)
    return df

list_to_pandas()

,City,Loc,Temp
0,Seven Springs,"40.0231,-79.2988",37
1,Marquette,"46.5179,-87.4034",28
2,Park City,"40.6452,-111.4927",27
3,South Lake Tahoe,"38.9387,-119.9810",40
4,Juneau,"58.3022,-134.41932",27
5,Glacier,"48.8087,-121.7031",34
